In [1]:
#%% Libraries
# # Standard Libraries
from os import path
from pathlib import Path
from pprint import pprint
import asyncio as asyncio

# External Libraries
from IPython.display import display
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from sklearn.linear_model import BayesianRidge as BR
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error as mse
import statistics as stats
import matplotlib.pylab as mlt

# Custom Libraries
from modules.shared.custom import split, getPI

In [2]:
# Just getting datasets
dataRoute = '../data/chembl/Additional_datasets/'
paths = ['data_CHEMBL313.csv', 'data_CHEMBL2637.csv', 'data_CHEMBL4124.csv']
dataSets = [pd.read_csv(dataRoute + path) for path in paths]
data = dataSets[0].sample(frac=1)
display(data)

,molecule_id,pXC50,b0001,b0002,b0003,b0004,b0005,b0006,b0007,b0008,...,b1015,b1016,b1017,b1018,b1019,b1020,b1021,b1022,b1023,b1024
2984,CHEMBL52324,5.967,1,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3382,CHEMBL67387,7.411,1,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1996,CHEMBL329517,5.139,1,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,0,0
2686,CHEMBL43512,5.796,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1130,CHEMBL210978,8.377,1,1,1,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,CHEMBL353910,7.600,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
376,CHEMBL133138,5.301,1,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1679,CHEMBL283473,5.420,1,0,1,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1863,CHEMBL305905,6.215,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
class Models:
    def __init__(self, m):
        self.models = m
    def fit(self, *args):
        self.trainedModels = [m.fit(*args) for m in self.models]
    def predict(self, *args):
        return np.mean([m.predict(*args) for m in self.trainedModels], axis=0)

def validate(y_true, y_false):
    return mse(y_true, y_false, sample_weight=y_true)

In [4]:
# Splitting up the data
X_known, Y_known, X_unknown, Y_unknown, X_test, Y_test = split(data, 160)

# Each testing stage can test 160 cases and up to 2000 samples will be tested
testSize = 160
maxSamples = len(Y_unknown)

# models to be used and pre-defining variables
models = Models([BR(), KNN(), RFR()])
predicitions: list
scores: list
smartScores: list
tree: KDTree

In [5]:
runningScore = []

for i in range(0, maxSamples, testSize):
    print(f'\r{i}.0 out of {maxSamples}', end="")
    models.fit(X_known, Y_known)
    Y_predict = models.predict(X_test)
    score = validate(Y_test, Y_predict)
    runningScore.append(score)


2880.0 out of 3014

In [6]:
runningScore


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]